# 1. Packages

In [1]:
import numpy as np
import pandas as pd 
import seaborn as sns

# Keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD, Adam, Adadelta, RMSprop
import keras.backend as K

# Train-Test
from sklearn.model_selection import train_test_split

# Scale
from sklearn.preprocessing import StandardScaler

# Classification Report
from sklearn.metrics import classification_report

Using TensorFlow backend.


# 2. Data

In [2]:
df = pd.read_csv("../input/data.csv")

# Remove Missing Values 
na = pd.notnull(df["Position"])
df = df[na]

df.head()

Unnamed: 0      ID               Name  Age  \
0           0  158023           L. Messi   31   
1           1   20801  Cristiano Ronaldo   33   
2           2  190871          Neymar Jr   26   
3           3  193080             De Gea   27   
4           4  192985       K. De Bruyne   27   

                                            Photo Nationality  \
0  https://cdn.sofifa.org/players/4/19/158023.png   Argentina   
1   https://cdn.sofifa.org/players/4/19/20801.png    Portugal   
2  https://cdn.sofifa.org/players/4/19/190871.png      Brazil   
3  https://cdn.sofifa.org/players/4/19/193080.png       Spain   
4  https://cdn.sofifa.org/players/4/19/192985.png     Belgium   

                                  Flag  Overall  Potential  \
0  https://cdn.sofifa.org/flags/52.png       94         94   
1  https://cdn.sofifa.org/flags/38.png       94         94   
2  https://cdn.sofifa.org/flags/54.png       92         93   
3  https://cdn.sofifa.org/flags/45.png       91         93   
4   https://cdn.sofifa.org/flags/7.png       91         92   

                  Club  ... Composure Marking StandingTackle  SlidingTackle  \
0         FC Barcelona  ...      96.0    33.0           28.0           26.0   
1             Juventus  ...      95.0    28.0           31.0           23.0   
2  Paris Saint-Germain  ...      94.0    27.0           24.0           33.0   
3    Manchester United  ...      68.0    15.0           21.0           13.0   
4      Manchester City  ...      88.0    68.0           58.0           51.0   

  GKDiving  GKHandling  GKKicking  GKPositioning GKReflexes Release Clause  
0      6.0        11.0       15.0           14.0        8.0        €226.5M  
1      7.0        11.0       15.0           14.0       11.0        €127.1M  
2      9.0         9.0       15.0           15.0       11.0        €228.1M  
3     90.0        85.0       87.0           88.0       94.0        €138.6M  
4     15.0        13.0        5.0           10.0       13.0        €196.4M  

[5 rows x 89 columns]

Position Category

In [3]:
forward = ["ST", "LW", "RW", "LF", "RF", "RS","LS", "CF"]
midfielder = ["CM","RCM","LCM", "CDM","RDM","LDM", "CAM", "LAM", "RAM", "RM", "LM"]
defender = ["CB", "RCB", "LCB", "LWB", "RWB", "LB", "RB"]

In [4]:
df.loc[df["Position"] == "GK", "Position"] = 0
df.loc[df["Position"].isin(defender), "Position"] = 1
df.loc[df["Position"].isin(midfielder), "Position"] = 2
df.loc[df["Position"].isin(forward), "Position"] = 3

In [5]:
df["Position"].value_counts()

2    6838
1    5866
3    3418
0    2025
Name: Position, dtype: int64

In [6]:
df["Position"].unique()

array([3, 0, 2, 1])

In [7]:
df = df[["Position", 'Finishing', 'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling',
       'Curve', 'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration',
       'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower',
       'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression',
       'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
       'Marking', 'StandingTackle', 'SlidingTackle', 'GKDiving', 'GKHandling',
       'GKKicking', 'GKPositioning', 'GKReflexes']]
df.head()

Position  Finishing  HeadingAccuracy  ShortPassing  Volleys  Dribbling  \
0         3       95.0             70.0          90.0     86.0       97.0   
1         3       94.0             89.0          81.0     87.0       88.0   
2         3       87.0             62.0          84.0     84.0       96.0   
3         0       13.0             21.0          50.0     13.0       18.0   
4         2       82.0             55.0          92.0     82.0       86.0   

   Curve  FKAccuracy  LongPassing  BallControl  ...  Penalties  Composure  \
0   93.0        94.0         87.0         96.0  ...       75.0       96.0   
1   81.0        76.0         77.0         94.0  ...       85.0       95.0   
2   88.0        87.0         78.0         95.0  ...       81.0       94.0   
3   21.0        19.0         51.0         42.0  ...       40.0       68.0   
4   85.0        83.0         91.0         91.0  ...       79.0       88.0   

   Marking  StandingTackle  SlidingTackle  GKDiving  GKHandling  GKKicking  \
0     33.0            28.0           26.0       6.0        11.0       15.0   
1     28.0            31.0           23.0       7.0        11.0       15.0   
2     27.0            24.0           33.0       9.0         9.0       15.0   
3     15.0            21.0           13.0      90.0        85.0       87.0   
4     68.0            58.0           51.0      15.0        13.0        5.0   

   GKPositioning  GKReflexes  
0           14.0         8.0  
1           14.0        11.0  
2           15.0        11.0  
3           88.0        94.0  
4           10.0        13.0  

[5 rows x 34 columns]

In [8]:
x = df.drop("Position", axis = 1)

from sklearn.preprocessing import StandardScaler
ss = StandardScaler()

x = pd.DataFrame(ss.fit_transform(x))

y = df["Position"]

x.head()

0         1         2         3         4         5         6   \
0  2.532391  1.018293  2.130190  2.434969  2.201010  2.491028  2.925359   
1  2.481180  2.111424  1.517765  2.491481  1.725114  1.838695  1.895584   
2  2.122700  0.558028  1.721906  2.321945  2.148132  2.219223  2.524891   
3 -1.666942 -1.800833 -0.591700 -1.690394 -1.976298 -1.422972 -1.365369   
4  1.866643  0.155296  2.266284  2.208922  1.619359  2.056139  2.296052   

         7         8         9   ...        23        24        25        26  \
0  2.236808  2.255005  1.767403  ...  1.684657  3.265669 -0.717896 -0.909470   
1  1.584271  2.135138  1.633444  ...  2.321491  3.178232 -0.969153 -0.770985   
2  1.649524  2.195072  1.968342  ...  2.066757  3.090795 -1.019405 -1.094116   
3 -0.112327 -0.981399 -0.509905  ... -0.544262  0.817436 -1.622423 -1.232601   
4  2.497823  1.955338  0.896668  ...  1.939390  2.566174  1.040907  0.475378   

         27        28        29        30        31        32  
0 -0.923850 -0.599889 -0.318983 -0.074713 -0.140266 -0.485159  
1 -1.064778 -0.543386 -0.318983 -0.074713 -0.140266 -0.318094  
2 -0.595016 -0.430379 -0.437259 -0.074713 -0.081569 -0.318094  
3 -1.534540  4.146377  4.057254  4.287959  4.203333  4.304052  
4  0.250555 -0.091360 -0.200706 -0.680640 -0.375055 -0.206717  

[5 rows x 33 columns]

In [9]:
y.head()

0    3
1    3
2    3
3    0
4    2
Name: Position, dtype: int64

In [10]:
from keras.utils.np_utils import to_categorical
y_cat = to_categorical(y)

y_cat[:10]

array([[0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.]], dtype=float32)

In [11]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x.values, y_cat,
                                                    test_size=0.2)

In [12]:
y.shape

(18147,)

In [13]:
x.shape

(18147, 33)

# 3. Deep Learning

In [14]:
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD, Adam, Adadelta, RMSprop

In [15]:
K.clear_session()
model = Sequential()
model.add(Dense(60, input_shape = (33,), activation = "relu"))
model.add(Dense(15, activation = "relu"))
model.add(Dense(4, activation = "softmax"))
model.compile(Adam(lr = 0.01), "categorical_crossentropy", metrics = ["accuracy"])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 60)                2040      
_________________________________________________________________
dense_2 (Dense)              (None, 15)                915       
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 64        
Total params: 3,019
Trainable params: 3,019
Non-trainable params: 0
_________________________________________________________________


In [16]:
60 * 33 + 60

2040

In [17]:
model.fit(x_train, y_train, verbose=1, epochs=10)

Epoch 1/10
14517/14517 [==============================] - 1s 83us/step - loss: 0.3395 - acc: 0.8556
Epoch 2/10
14517/14517 [==============================] - 1s 47us/step - loss: 0.3062 - acc: 0.8723
Epoch 3/10
14517/14517 [==============================] - 1s 47us/step - loss: 0.2986 - acc: 0.8748
Epoch 4/10
14517/14517 [==============================] - 1s 48us/step - loss: 0.2922 - acc: 0.8764
Epoch 5/10
14517/14517 [==============================] - 1s 48us/step - loss: 0.2903 - acc: 0.8781
Epoch 6/10
14517/14517 [==============================] - 1s 46us/step - loss: 0.2866 - acc: 0.8808
Epoch 7/10
14517/14517 [==============================] - 1s 47us/step - loss: 0.2844 - acc: 0.8832
Epoch 8/10
14517/14517 [==============================] - 1s 47us/step - loss: 0.2795 - acc: 0.8856
Epoch 9/10
14517/14517 [==============================] - 1s 45us/step - loss: 0.2753 - acc: 0.8850
Epoch 10/10
14517/14517 [==============================] - 1s 47us/step - loss: 0.2747 - acc: 0.8862

In [18]:
y_pred_class = model.predict_classes(x_test)

In [19]:
from sklearn.metrics import confusion_matrix

In [20]:
y_pred = model.predict(x_test)

In [21]:
y_test_class = np.argmax(y_test, axis=1)

In [22]:
y_test_class

array([3, 3, 3, ..., 1, 3, 1])

In [23]:
confusion_matrix(y_test_class, y_pred_class)

array([[ 407,    0,    0,    0],
       [   0,  998,  187,    1],
       [   0,   59, 1093,  176],
       [   0,    4,   97,  608]])

In [24]:
from sklearn.metrics import classification_report
print(classification_report(y_test_class, y_pred_class))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       407
           1       0.94      0.84      0.89      1186
           2       0.79      0.82      0.81      1328
           3       0.77      0.86      0.81       709

    accuracy                           0.86      3630
   macro avg       0.88      0.88      0.88      3630
weighted avg       0.86      0.86      0.86      3630

